In [1]:
################################
#
# Dataset: https://www.kaggle.com/tevecsystems/retail-sales-forecasting
#
# The data contains the sales and price of a product at the end of each day
# We will forecast the sales for next day, given the sales for the last few days
# Using LSTMs cells and RNN
#
# Author: Anurag Dwarakanath
################################

In [2]:
import tensorflow as tf
import pandas
import numpy as np

In [24]:
# Constants
DATAFILE_TRAIN = "/home/ubuntu/anurag/rnn/mock_kaggle_edit_train.csv" 

DATAFILE_VALIDATE = "/home/ubuntu/anurag/rnn/mock_kaggle_edit_validate.csv"

# The original data is here: https://www.kaggle.com/tevecsystems/retail-sales-forecasting
# The original data was simplified to consider only 'date', 'sales', 'price'
# However, we will consider only 'sales' & 'date'

TRAINED_MODEL_PATH = '/home/ubuntu/anurag/rnn/savedModel'

TIME_STEPS = 10 # i.e. look at the past 10 days and forecast
NUMBER_OF_DAYS_TO_FORECAST = 1 # for now we will only forecast the next day's sales

BATCH_SIZE=100

NUM_EPOCHS=100

LSTM_UNITS = 250

TENSORBOARD_LOGDIR = '/home/ubuntu/anurag/rnn/tensorboard_log'

In [25]:
#ingest the data
data_train = pandas.read_csv(DATAFILE_TRAIN) # data will be an array of dict. E.g. data['sales'][0]
data_validate = pandas.read_csv(DATAFILE_VALIDATE)

In [26]:
# let's print the first 5 rows of the data
data_train.head()

,date,sales,price
0,1/1/2014,0,1.29
1,1/2/2014,70,1.29
2,1/3/2014,59,1.29
3,1/4/2014,93,1.29
4,1/5/2014,96,1.29


In [27]:
# Create the training & validation data

numTrainingData = len(data_train)
numValidationData = len(data_validate)

trainingData_date = data_train['date'][0:numTrainingData]
trainingData_sales = data_train['sales'][0:numTrainingData]
trainindData_price = data_train['price'][0:numTrainingData]

validationData_date = data_validate['date'][0: numValidationData]
validationData_sales = data_validate['sales'][0: numValidationData]
validationData_price = data_validate['price'][0: numValidationData]

In [45]:
trainingData_sales.head()

0     0
1    70
2    59
3    93
4    96
Name: sales, dtype: int64

In [29]:
print(len(trainingData_sales))
print(len(validationData_sales))

750
187


In [30]:
# Normalise the training and validation data.
# We use the (x - min)/(range) to normalise, where range = (max - min)
# This will put the values between 0 & 1
trainingData_sales_min = min(trainingData_sales)
trainingData_sales_max = max(trainingData_sales)
trainingData_sales_range = trainingData_sales_max - trainingData_sales_min
trainingData_sales_normalised = [(i - trainingData_sales_min)/trainingData_sales_range for i in trainingData_sales]

validationData_sales_normalised = [(i - trainingData_sales_min)/trainingData_sales_range for i in validationData_sales]
# Note the validation data uses the max & min values of Training Data

In [31]:
print('Min:', trainingData_sales_min)
print('Range:', trainingData_sales_max - trainingData_sales_min)

Min: 0
Range: 542


In [32]:
# we will create the set of sequences
trainingDataSequence_sales = np.zeros(shape=(len(trainingData_sales)-TIME_STEPS-NUMBER_OF_DAYS_TO_FORECAST+1, TIME_STEPS, 1))
targetDataSequence_sales = np.zeros(shape=(len(trainingData_sales)-TIME_STEPS-NUMBER_OF_DAYS_TO_FORECAST+1, NUMBER_OF_DAYS_TO_FORECAST))
start = 0
for i in range(TIME_STEPS, len(trainingData_sales)-NUMBER_OF_DAYS_TO_FORECAST+1):
    trainingDataSequence_sales[start,:,0]=trainingData_sales_normalised[start:i]
    targetDataSequence_sales[start] = trainingData_sales_normalised[i:i+NUMBER_OF_DAYS_TO_FORECAST]
    start=start+1

## MT: make sure the sequences do not miss out any data
# maybe create predictable data (1, 2, 3, 4) then put completely rubbish data (23434) at the end.. this should be picked up in training


## The format of the data sequences is as follows:

trainingDataSequence_sales = \[sales_day1 sales_day2 sales_day3 ... sales_day10\] 

targetDataSequence_sales = \[sales_day11\]

Although the trainingDataSequence_sales can be captured through a 2D array, we use 3D (i.e. shape = (num of sequence, time steps, 1) ) because the tensor input for the LSTM cell expects a 3D input. The third dimension is the *num of features*  of each data point (in our case, it is a single dimension).

In [33]:
# Print the first few sequences of the sales that have been created
[trainingDataSequence_sales[i, :, 0] for i in range(3)]

[array([0.        , 0.12915129, 0.10885609, 0.17158672, 0.17712177,
        0.26752768, 0.3302583 , 0.59225092, 0.23062731, 0.16236162]),
 array([0.12915129, 0.10885609, 0.17158672, 0.17712177, 0.26752768,
        0.3302583 , 0.59225092, 0.23062731, 0.16236162, 0.34686347]),
 array([0.10885609, 0.17158672, 0.17712177, 0.26752768, 0.3302583 ,
        0.59225092, 0.23062731, 0.16236162, 0.34686347, 0.22324723])]

In [34]:
#Print the first few target sales
[targetDataSequence_sales[i] for i in range(3)]

[array([0.34686347]), array([0.22324723]), array([0.24723247])]

###### Observe that the first sales target (0.34686347) becomes the last day sales of the next triainingDataSequence

i.e. targetDataSequence_sales\[i\] = targetDataSequence_sales\[i+1,TIME_STEPS-1,0\]

In [35]:
# since we are making each sequence independent to each other,
# the order of the sequences can be shuffled.
# Note that we need to ensure the target and the train sequences are in the same order
a = np.arange(len(targetDataSequence_sales))
np.random.shuffle(a)
trainingDataSequence_sales_shuffle = np.zeros(shape=(len(trainingData_sales)-TIME_STEPS-NUMBER_OF_DAYS_TO_FORECAST+1, TIME_STEPS, 1))
targetDataSequence_sales_shuffle = np.zeros(shape=(len(trainingData_sales)-TIME_STEPS-NUMBER_OF_DAYS_TO_FORECAST+1, NUMBER_OF_DAYS_TO_FORECAST))

loc=0
for i in a:
    trainingDataSequence_sales_shuffle[loc] = trainingDataSequence_sales[i]
    targetDataSequence_sales_shuffle[loc] = targetDataSequence_sales[i]
    loc+=1

trainingDataSequence_sales = trainingDataSequence_sales_shuffle
targetDataSequence_sales = targetDataSequence_sales_shuffle

In [36]:
# Build the seqeunces for validation
validationDataSequence_sales = np.zeros(shape=(len(validationData_sales) - TIME_STEPS - NUMBER_OF_DAYS_TO_FORECAST + 1, TIME_STEPS, 1 ))
validationDataSequence_sales_target = np.zeros(shape=(len(validationData_sales) - TIME_STEPS - NUMBER_OF_DAYS_TO_FORECAST + 1, NUMBER_OF_DAYS_TO_FORECAST))

start = 0
for i in range(TIME_STEPS, len(validationData_sales)-NUMBER_OF_DAYS_TO_FORECAST + 1):
    validationDataSequence_sales[start, :, 0] = validationData_sales_normalised[start:i]
    validationDataSequence_sales_target[start] = validationData_sales_normalised[i:i+NUMBER_OF_DAYS_TO_FORECAST]
    start+=1

In [37]:
#build the RNN model
tf.reset_default_graph()

inputSequencePlaceholder = tf.placeholder(dtype=tf.float32, shape=(None, TIME_STEPS, 1), name='inputSequencePlaceholder')
targetPlaceholder = tf.placeholder(dtype=tf.float32, shape=(None, NUMBER_OF_DAYS_TO_FORECAST), name='targetPlaceholder')

#cell = tf.contrib.cudnn_rnn.CudnnLSTM(num_layers=1, num_units=5, name='LSTM_cell')
cell = tf.nn.rnn_cell.LSTMCell(num_units=LSTM_UNITS, name='LSTM_cell')


output, state = tf.nn.dynamic_rnn(cell=cell, inputs=inputSequencePlaceholder, dtype=tf.float32)
#output is of shape (Batch, time, num_units)

lastCellOutput = output[:, -1,:]

In [38]:
print('output:',output)
print('state:', state)
print('lastCellOutput:', lastCellOutput)

output: Tensor("rnn/transpose_1:0", shape=(?, 10, 250), dtype=float32)
state: LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 250) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 250) dtype=float32>)
lastCellOutput: Tensor("strided_slice:0", shape=(?, 250), dtype=float32)


## Understanding the type and shape of **output** & **state**
the RNN gives two outputs for every sequence of input: a) output & b) state.

**output** is the per time step output (i.e. output from each time step and which is represented as a_t in our notes). Thus it has a shape of (Batch, 10, 25). We are interested only in the final time step output (i.e. Batch, 9, 250) and this is captured by lastCellOutput

the **state** is the value of the internal parameters of the LSTM that can be propogated to the next sequence if need be. The state is of type LSTMStateTuple which contains two tensors of shape (batch, 250) each.

In [39]:
#build the output layer from the RNN outputs
weights = tf.Variable(initial_value=tf.truncated_normal(shape=(LSTM_UNITS, NUMBER_OF_DAYS_TO_FORECAST)))
bias = tf.Variable(initial_value=tf.ones(shape=(NUMBER_OF_DAYS_TO_FORECAST)))

forecast = tf.add(x=tf.matmul(a=lastCellOutput, b=weights),y= bias, name='forecast_normalised_scale')

#tf.summary.scalar(tensor=forecast, name='forecast_normalised_scale')

# bring back the forecast to the original scale
forecast_originalScale = tf.add(x = forecast * trainingData_sales_range ,y = trainingData_sales_min, name='forecast_original_scale')

In [40]:
print(forecast)
print(forecast_originalScale)

Tensor("forecast_normalised_scale:0", shape=(?, 1), dtype=float32)
Tensor("forecast_original_scale:0", shape=(?, 1), dtype=float32)


the **forecast** is the layer that computes the final forecast. this is the y_hat in our notes. **forecast** will have a shape of (Batch, 1)

In [41]:
loss = tf.reduce_mean(tf.squared_difference(x=forecast, y=targetPlaceholder), name='loss_comp')

tf.summary.scalar(tensor=loss, name='loss')

<tf.Tensor 'loss:0' shape=() dtype=string>

In [42]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
minimize_step = optimizer.minimize(loss)

In [43]:
# print([n.name for n in tf.get_default_graph().as_graph_def().node])

In [44]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    #Get the tensorboard writer
    tensorboard_writer = tf.summary.FileWriter(TENSORBOARD_LOGDIR, sess.graph)
    
    #merge all the summary ops
    all_summary_ops = tf.summary.merge_all()
    
    #start the training
    numSteps=0
    for e in range(NUM_EPOCHS):
        print('starting training for epoch:', e+1)
        
        startLocation=0
        iteration=0
        for iteration in range(int(len(targetDataSequence_sales)/BATCH_SIZE)):
            print('epoch:', e+1, ' iteration:', iteration+1)
            trainingBatchInput = trainingDataSequence_sales[startLocation:startLocation+BATCH_SIZE, :, :]
            trainingBatchTarget = targetDataSequence_sales[startLocation:startLocation+BATCH_SIZE]
            
            _, lsBatch, forecastBatch, forecastBatch_originalScale, summary_values = sess.run([minimize_step, loss, forecast, forecast_originalScale, all_summary_ops], feed_dict={inputSequencePlaceholder: trainingBatchInput,
                                                                                            targetPlaceholder: trainingBatchTarget})
            
            tensorboard_writer.add_summary(summary_values, numSteps)
            numSteps+=1
            
            if ((iteration+1) % 1 == 0):
                print('got a loss of:', lsBatch)
                print('the forecast of first 5 normalised are:', forecastBatch[0:5])
                print('while the actuals were normalised     :', trainingBatchTarget[0:5])
                print('the forecast of first 5 orignal scale are:', forecastBatch_originalScale[0:5])
                print('while the actuals were original scale     :', trainingBatchTarget[0:5]* trainingData_sales_range + trainingData_sales_min)
            
            startLocation+=BATCH_SIZE
        
        #pick up the training data that does not fit into BATCH_SIZE
        if (len(targetDataSequence_sales) > startLocation):
            print('epoch:', e+1, ' iteration:', iteration+1)
            trainingBatchInput = trainingDataSequence_sales[startLocation:len(targetDataSequence_sales), :, :]
            trainingBatchTarget = targetDataSequence_sales[startLocation:len(targetDataSequence_sales)]
            
            _, lsBatch, forecastBatch, forecastBatch_originalScale = sess.run([minimize_step, loss, forecast, forecast_originalScale], feed_dict={inputSequencePlaceholder: trainingBatchInput,
                                                                                            targetPlaceholder: trainingBatchTarget})
            
            print('got a loss of:', lsBatch)
            print('the forecast of first 5 normalised are:', forecastBatch[0:5])
            print('while the actuals were normalised     :', trainingBatchTarget[0:5])
            print('the forecast of first 5 orignal scale are:', forecastBatch_originalScale[0:5])
            print('while the actuals were original scale     :', trainingBatchTarget[0:5]* trainingData_sales_range + trainingData_sales_min)
    
    
        #end of 1 epoch. Perform validation
        totalValidationLoss = 0
        startLocation=0
        print('starting validation')
        for iter in range(len(validationDataSequence_sales)//BATCH_SIZE):
            validationBatchInput = validationDataSequence_sales[startLocation: startLocation+BATCH_SIZE, :, :]
            validationBatchTarget = validationDataSequence_sales_target[startLocation: startLocation+BATCH_SIZE]
            
            validationLsBatch, validationForecastBatch, validationForecastBatch_originalScale = sess.run([loss, forecast, forecast_originalScale], feed_dict={inputSequencePlaceholder: validationBatchInput,
                                                                                              targetPlaceholder: validationBatchTarget})
            
           
            startLocation+=BATCH_SIZE
            totalValidationLoss+= validationLsBatch
            
            print('first five predictions:', validationForecastBatch[0:5])
            print('first five actuals    :', validationBatchTarget[0:5])
            print('the forecast of first 5 orignal scale are:', validationForecastBatch_originalScale[0:5])
            print('while the actuals were original scale     :', validationBatchTarget[0:5]* trainingData_sales_range + trainingData_sales_min)
    
        
        if(startLocation < len(validationDataSequence_sales)):
            validationBatchInput = validationDataSequence_sales[startLocation: len(validationDataSequence_sales)]
            validationBatchTarget = validationDataSequence_sales_target[startLocation: len(validationDataSequence_sales)]
            
            validationLsBatch, validationForecastBatch = sess.run([loss, forecast], feed_dict={inputSequencePlaceholder: validationBatchInput,
                                                                                              targetPlaceholder: validationBatchTarget})
            
            totalValidationLoss+= validationLsBatch
        
        #validation completed.
        print('Validation completed after epoch:',e+1 ,'. Total validation loss:', totalValidationLoss)
        
    # Save the model
    print('----------- Saving Model')
    tf.saved_model.simple_save(sess, export_dir=TRAINED_MODEL_PATH, 
                    inputs={"inputSequencePlaceholder": inputSequencePlaceholder, "targetPlaceholder":targetPlaceholder}, 
                    outputs={"loss": loss, "forecast_originalScale": forecast_originalScale})
    print('saved model to:',TRAINED_MODEL_PATH )
    
print('----------- Finis')
        
            
    

    

starting training for epoch: 1
epoch: 1  iteration: 1
got a loss of: 0.75377196
the forecast of first 5 normalised are: [[0.99473137]
 [1.0000477 ]
 [0.9990285 ]
 [0.99471855]
 [0.9996665 ]]
while the actuals were normalised     : [[0.1199262 ]
 [0.07380074]
 [0.28413284]
 [0.        ]
 [0.06826568]]
the forecast of first 5 orignal scale are: [[539.1444 ]
 [542.0258 ]
 [541.47345]
 [539.13745]
 [541.8193 ]]
while the actuals were original scale     : [[ 65.]
 [ 40.]
 [154.]
 [  0.]
 [ 37.]]
epoch: 1  iteration: 2
got a loss of: 6798.3623
the forecast of first 5 normalised are: [[-82.30445]
 [-82.31001]
 [-82.30894]
 [-82.30746]
 [-82.30726]]
while the actuals were normalised     : [[0.19741697]
 [0.        ]
 [0.08118081]
 [0.11439114]
 [0.23800738]]
the forecast of first 5 orignal scale are: [[-44609.01 ]
 [-44612.027]
 [-44611.445]
 [-44610.64 ]
 [-44610.535]]
while the actuals were original scale     : [[107.]
 [  0.]
 [ 44.]
 [ 62.]
 [129.]]
epoch: 1  iteration: 3
got a loss of: 70

while the actuals were original scale     : [[ 65.]
 [ 40.]
 [154.]
 [  0.]
 [ 37.]]
epoch: 4  iteration: 2
got a loss of: 0.016394451
the forecast of first 5 normalised are: [[0.22375023]
 [0.10556412]
 [0.11208022]
 [0.18609416]
 [0.14489615]]
while the actuals were normalised     : [[0.19741697]
 [0.        ]
 [0.08118081]
 [0.11439114]
 [0.23800738]]
the forecast of first 5 orignal scale are: [[121.27263 ]
 [ 57.21575 ]
 [ 60.74748 ]
 [100.86304 ]
 [ 78.533714]]
while the actuals were original scale     : [[107.]
 [  0.]
 [ 44.]
 [ 62.]
 [129.]]
epoch: 4  iteration: 3
got a loss of: 0.01706857
the forecast of first 5 normalised are: [[0.10662365]
 [0.04147005]
 [0.04802883]
 [0.09224832]
 [0.04152346]]
while the actuals were normalised     : [[0.16236162]
 [0.        ]
 [0.1199262 ]
 [0.22693727]
 [0.        ]]
the forecast of first 5 orignal scale are: [[57.790016]
 [22.476768]
 [26.031624]
 [49.99859 ]
 [22.505714]]
while the actuals were original scale     : [[ 88.]
 [  0.]
 [ 6

got a loss of: 0.014954215
the forecast of first 5 normalised are: [[0.19418216]
 [0.10093117]
 [0.105281  ]
 [0.16798627]
 [0.13488364]]
while the actuals were normalised     : [[0.19741697]
 [0.        ]
 [0.08118081]
 [0.11439114]
 [0.23800738]]
the forecast of first 5 orignal scale are: [[105.24673 ]
 [ 54.704693]
 [ 57.062298]
 [ 91.04856 ]
 [ 73.10693 ]]
while the actuals were original scale     : [[107.]
 [  0.]
 [ 44.]
 [ 62.]
 [129.]]
epoch: 7  iteration: 3
got a loss of: 0.019003548
the forecast of first 5 normalised are: [[0.16908443]
 [0.11495316]
 [0.11939871]
 [0.15717578]
 [0.11497641]]
while the actuals were normalised     : [[0.16236162]
 [0.        ]
 [0.1199262 ]
 [0.22693727]
 [0.        ]]
the forecast of first 5 orignal scale are: [[91.64376]
 [62.30461]
 [64.7141 ]
 [85.18927]
 [62.31721]]
while the actuals were original scale     : [[ 88.]
 [  0.]
 [ 65.]
 [123.]
 [  0.]]
epoch: 7  iteration: 4
got a loss of: 0.021131651
the forecast of first 5 normalised are: [

got a loss of: 0.016819535
the forecast of first 5 normalised are: [[0.1459111 ]
 [0.10339415]
 [0.10778189]
 [0.13764822]
 [0.10342586]]
while the actuals were normalised     : [[0.16236162]
 [0.        ]
 [0.1199262 ]
 [0.22693727]
 [0.        ]]
the forecast of first 5 orignal scale are: [[79.08382 ]
 [56.03963 ]
 [58.41778 ]
 [74.60534 ]
 [56.056816]]
while the actuals were original scale     : [[ 88.]
 [  0.]
 [ 65.]
 [123.]
 [  0.]]
epoch: 10  iteration: 4
got a loss of: 0.017748686
the forecast of first 5 normalised are: [[0.10717392]
 [0.10211849]
 [0.12748313]
 [0.1363275 ]
 [0.10227036]]
while the actuals were normalised     : [[0.11808118]
 [0.        ]
 [0.06457565]
 [0.00184502]
 [0.02398524]]
the forecast of first 5 orignal scale are: [[58.088264]
 [55.34822 ]
 [69.095856]
 [73.88951 ]
 [55.43054 ]]
while the actuals were original scale     : [[64.]
 [ 0.]
 [35.]
 [ 1.]
 [13.]]
epoch: 10  iteration: 5
got a loss of: 0.0212436
the forecast of first 5 normalised are: [[0.14

got a loss of: 0.014242316
the forecast of first 5 normalised are: [[0.17558563]
 [0.12092686]
 [0.12471271]
 [0.16223764]
 [0.14390278]]
while the actuals were normalised     : [[0.19741697]
 [0.        ]
 [0.08118081]
 [0.11439114]
 [0.23800738]]
the forecast of first 5 orignal scale are: [[95.16741 ]
 [65.54236 ]
 [67.594284]
 [87.9328  ]
 [77.99531 ]]
while the actuals were original scale     : [[107.]
 [  0.]
 [ 44.]
 [ 62.]
 [129.]]
epoch: 13  iteration: 3
got a loss of: 0.017338658
the forecast of first 5 normalised are: [[0.15921271]
 [0.12343705]
 [0.12916279]
 [0.15454948]
 [0.12349701]]
while the actuals were normalised     : [[0.16236162]
 [0.        ]
 [0.1199262 ]
 [0.22693727]
 [0.        ]]
the forecast of first 5 orignal scale are: [[86.29329 ]
 [66.90288 ]
 [70.00623 ]
 [83.765816]
 [66.93538 ]]
while the actuals were original scale     : [[ 88.]
 [  0.]
 [ 65.]
 [123.]
 [  0.]]
epoch: 13  iteration: 4
got a loss of: 0.01750942
the forecast of first 5 normalised are: 

...

epoch: 89  iteration: 7
got a loss of: 0.018423324
the forecast of first 5 normalised are: [[0.18447006]
 [0.13857436]
 [0.23333871]
 [0.11395323]
 [0.0932951 ]]
while the actuals were normalised     : [[0.11439114]
 [0.09225092]
 [0.25276753]
 [0.        ]
 [0.        ]]
the forecast of first 5 orignal scale are: [[ 99.98277 ]
 [ 75.10731 ]
 [126.46958 ]
 [ 61.762653]
 [ 50.56594 ]]
while the actuals were original scale     : [[ 62.]
 [ 50.]
 [137.]
 [  0.]
 [  0.]]
epoch: 89  iteration: 7
got a loss of: 0.008806742
the forecast of first 5 normalised are: [[0.19921458]
 [0.25093663]
 [0.20450783]
 [0.19664693]
 [0.22480857]]
while the actuals were normalised     : [[0.19741697]
 [0.17343173]
 [0.099631  ]
 [0.20479705]
 [0.17896679]]
the forecast of first 5 orignal scale are: [[107.974304]
 [136.00764 ]
 [110.84325 ]
 [106.582634]
 [121.846245]]
while the actuals were original scale     : [[107.]
 [ 94.]
 [ 54.]
 [111.]
 [ 97.]]
starting validation
first five predictions: [[0.15700877

got a loss of: 0.01334485
the forecast of first 5 normalised are: [[0.17938542]
 [0.07314205]
 [0.10365319]
 [0.04637027]
 [0.14462078]]
while the actuals were normalised     : [[0.61808118]
 [0.0498155 ]
 [0.20479705]
 [0.14206642]
 [0.17343173]]
the forecast of first 5 orignal scale are: [[97.2269  ]
 [39.64299 ]
 [56.18003 ]
 [25.132685]
 [78.38446 ]]
while the actuals were original scale     : [[335.]
 [ 27.]
 [111.]
 [ 77.]
 [ 94.]]
epoch: 92  iteration: 7
got a loss of: 0.0193766
the forecast of first 5 normalised are: [[0.14719105]
 [0.0999105 ]
 [0.19782996]
 [0.07180452]
 [0.05218744]]
while the actuals were normalised     : [[0.11439114]
 [0.09225092]
 [0.25276753]
 [0.        ]
 [0.        ]]
the forecast of first 5 orignal scale are: [[ 79.77755 ]
 [ 54.15149 ]
 [107.22384 ]
 [ 38.918053]
 [ 28.285595]]
while the actuals were original scale     : [[ 62.]
 [ 50.]
 [137.]
 [  0.]
 [  0.]]
epoch: 92  iteration: 7
got a loss of: 0.009575805
the forecast of first 5 normalised ar

got a loss of: 0.018322652
the forecast of first 5 normalised are: [[0.15520382]
 [0.1056819 ]
 [0.21275187]
 [0.07724547]
 [0.05857694]]
while the actuals were normalised     : [[0.11439114]
 [0.09225092]
 [0.25276753]
 [0.        ]
 [0.        ]]
the forecast of first 5 orignal scale are: [[ 84.12047 ]
 [ 57.279587]
 [115.31151 ]
 [ 41.867046]
 [ 31.748703]]
while the actuals were original scale     : [[ 62.]
 [ 50.]
 [137.]
 [  0.]
 [  0.]]
epoch: 95  iteration: 7
got a loss of: 0.009380517
the forecast of first 5 normalised are: [[0.20285988]
 [0.27565074]
 [0.20688593]
 [0.2011776 ]
 [0.2319572 ]]
while the actuals were normalised     : [[0.19741697]
 [0.17343173]
 [0.099631  ]
 [0.20479705]
 [0.17896679]]
the forecast of first 5 orignal scale are: [[109.95006]
 [149.4027 ]
 [112.13218]
 [109.03825]
 [125.7208 ]]
while the actuals were original scale     : [[107.]
 [ 94.]
 [ 54.]
 [111.]
 [ 97.]]
starting validation
first five predictions: [[0.15848339]
 [0.17118835]
 [0.16165447]

got a loss of: 0.008812735
the forecast of first 5 normalised are: [[0.1994586 ]
 [0.26744223]
 [0.20359242]
 [0.19883978]
 [0.22777426]]
while the actuals were normalised     : [[0.19741697]
 [0.17343173]
 [0.099631  ]
 [0.20479705]
 [0.17896679]]
the forecast of first 5 orignal scale are: [[108.10656 ]
 [144.95369 ]
 [110.34709 ]
 [107.771164]
 [123.45365 ]]
while the actuals were original scale     : [[107.]
 [ 94.]
 [ 54.]
 [111.]
 [ 97.]]
starting validation
first five predictions: [[0.15620553]
 [0.16873801]
 [0.1590035 ]
 [0.11163092]
 [0.17942262]]
first five actuals    : [[0.15682657]
 [0.14391144]
 [0.13099631]
 [0.20848708]
 [0.25461255]]
the forecast of first 5 orignal scale are: [[84.6634  ]
 [91.456   ]
 [86.17989 ]
 [60.503956]
 [97.247055]]
while the actuals were original scale     : [[ 85.]
 [ 78.]
 [ 71.]
 [113.]
 [138.]]
Validation completed after epoch: 98 . Total validation loss: 0.04394811391830444
starting training for epoch: 99
epoch: 99  iteration: 1
got a loss

INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/ubuntu/anurag/rnn/savedModel/saved_model.pb
saved model to: /home/ubuntu/anurag/rnn/savedModel
----------- Finis
